# Process Illinois County-level & Zipcode-level Data from IDPH

In [1]:
import pandas as pd
import json
import numpy as np
import geopandas as gpd
import shapely.wkt
import requests
from urllib.request import urlopen
from datetime import datetime, timedelta

URL preparation

In [2]:
# idph county geometry
county_geometry_url = "./idph_county_geometry.geojson"
county_geometry = gpd.read_file(county_geometry_url)
county_geometry

,id,population,geometry
0,Chicago,2693976,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ..."
1,McHenry,308570,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
2,Boone,53577,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
3,Ogle,50923,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
4,Will,692310,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
...,...,...,...
99,Marion,37620,"POLYGON ((-89.13844 38.73633, -89.13847 38.721..."
100,Union,16841,"POLYGON ((-89.04143 37.59650, -89.06017 37.597..."
101,Pope,4212,"POLYGON ((-88.70860 37.59926, -88.70876 37.584..."
102,Illinois,12671821,"POLYGON ((-89.64923 39.78269, -89.64923 39.779..."


Download data

Zipcode Data generation

County Data generation


In [3]:
county_geometry.id.unique().size

104

In [4]:
# IDPH data started from 03/17/2020
start_date = '03/17/2020'

# Get last update date from IDPH API
last_update_date_json = requests.get("https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetCountyHistorical?countyName=Illinois").json()
last_update_date = last_update_date_json['values'][-1]['ReportDate'][0:10]


# Get JSON file for each county in GeoJson file and append them all together
county_history_list = []
for county in county_geometry['id']:
    county_data_url = "https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetCountyHistorical?countyName=" + county
    d = requests.get(county_data_url).json()
    county_history_list = county_history_list + d['values']

# Generate dataframe
county_history_df = pd.DataFrame(county_history_list)
county_history_df

,ReportDate,CountyName,CumulativeCases,CasesChange,TotalTested,TotalTestedChange,Deaths,DeathsChange
0,2020-03-17T00:00:00,Chicago,0,0,0,0,0,0
1,2020-03-18T00:00:00,Chicago,0,0,0,0,0,0
2,2020-03-19T00:00:00,Chicago,0,0,0,0,0,0
3,2020-03-20T00:00:00,Chicago,0,0,0,0,0,0
4,2020-03-21T00:00:00,Chicago,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
62603,2021-11-04T00:00:00,Cook,316620,729,6362747,30626,5404,5
62604,2021-11-05T00:00:00,Cook,317108,488,6402007,39260,5408,4
62605,2021-11-06T00:00:00,Cook,317623,515,6437829,35822,5411,3
62606,2021-11-07T00:00:00,Cook,318051,428,6473328,35499,5413,2


In [5]:
# Change the date format and column names
county_history_df['ReportDate'] = county_history_df['ReportDate'].apply(lambda x: x[0:10])
county_history = county_history_df.rename(columns={"ReportDate":"date", 
                                                   "CountyName": "County", 
                                                   "TotalTested": "total_tested",
                                                  "CumulativeCases":"confirmed_cases",
                                                  "Deaths":"deaths"})
county_history

,date,County,confirmed_cases,CasesChange,total_tested,TotalTestedChange,deaths,DeathsChange
0,2020-03-17,Chicago,0,0,0,0,0,0
1,2020-03-18,Chicago,0,0,0,0,0,0
2,2020-03-19,Chicago,0,0,0,0,0,0
3,2020-03-20,Chicago,0,0,0,0,0,0
4,2020-03-21,Chicago,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
62603,2021-11-04,Cook,316620,729,6362747,30626,5404,5
62604,2021-11-05,Cook,317108,488,6402007,39260,5408,4
62605,2021-11-06,Cook,317623,515,6437829,35822,5411,3
62606,2021-11-07,Cook,318051,428,6473328,35499,5413,2


In [6]:
## Eliminate data from last update date or unassigned
#county_history = county_history[county_history['date'] != last_update_date]
county_history = county_history[county_history['County'] != 'Unassigned']
county_history

,date,County,confirmed_cases,CasesChange,total_tested,TotalTestedChange,deaths,DeathsChange
0,2020-03-17,Chicago,0,0,0,0,0,0
1,2020-03-18,Chicago,0,0,0,0,0,0
2,2020-03-19,Chicago,0,0,0,0,0,0
3,2020-03-20,Chicago,0,0,0,0,0,0
4,2020-03-21,Chicago,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
62603,2021-11-04,Cook,316620,729,6362747,30626,5404,5
62604,2021-11-05,Cook,317108,488,6402007,39260,5408,4
62605,2021-11-06,Cook,317623,515,6437829,35822,5411,3
62606,2021-11-07,Cook,318051,428,6473328,35499,5413,2


In [7]:
# Create pivot tables
county_pivot = pd.pivot_table(county_history, index=['County'],columns=['date'])
county_pivot = county_pivot.replace(np.nan, 0).astype(int)

county_cases = county_pivot['confirmed_cases']
county_deaths = county_pivot['deaths']
county_tested = county_pivot['total_tested']

In [8]:
# # Add missing 03/23 data with 03/22
# county_cases['2020-03-23'] = county_cases['2020-03-22']
# county_deaths['2020-03-23'] = county_deaths['2020-03-22']
# county_tested['2020-03-23'] = county_tested['2020-03-22']
# county_tested

In [9]:
# Get date information
date = county_cases.columns.tolist()
date.sort()
dt_first = date[0]
dt_today = date[-1]
dt_yesterday = date[-2]

In [10]:
# Define function for finding missing dates
def find_missing_date(date):
    missing_dates = []
    dt_range = datetime.strptime(date[-1], "%Y-%m-%d") - datetime.strptime(date[0], "%Y-%m-%d")
    if len(date) != dt_range.days + 1:
        print('Alert! Missing Date or Redundant date')
        for x in range(len(date) - 1):
            if (datetime.strptime(date[x + 1], "%Y-%m-%d") - datetime.strptime(date[x], "%Y-%m-%d")).days != 1:
                missing_date = datetime.strptime(date[x], "%Y-%m-%d") + timedelta(days=1)
                print('Missing Date:')
                print(missing_date)
                missing_dates.append(missing_date)
    return missing_dates

In [11]:
# Looking for missing dates
missing = find_missing_date(date)

Alert! Missing Date or Redundant date
Missing Date:
2020-03-23 00:00:00


In [12]:
# Fill in missing data for dates
for d_missing in missing:
   d_yesterday = d_missing - timedelta(days=1)
   d_key_yesterday = d_yesterday.strftime("%Y-%m-%d")
   d_key_missing = d_missing.strftime("%Y-%m-%d")
   county_cases[d_key_missing] = county_cases[d_key_yesterday]
   county_deaths[d_key_missing] = county_deaths[d_key_yesterday]
   county_tested[d_key_missing] = county_tested[d_key_yesterday]

/var/miniconda3/envs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/var/miniconda3/envs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/var/miniconda3/envs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [13]:
# Check missing dates again
date = county_cases.columns.tolist()
date.sort()
missings = find_missing_date(date)
if len(missings)> 0:
    raise Exception("Missing dates: {}".format(missings))

In [14]:
# Get date information
date = county_cases.columns.tolist()
date.sort()
dt_first = date[0]
dt_today = date[-1]
dt_yesterday = date[-2]

In [15]:
# Reorder date
county_cases = county_cases[date]
county_deaths = county_deaths[date]
county_tested = county_tested[date]

In [16]:
# Get case time series
cases_ts = county_cases.values.tolist()
deaths_ts = county_deaths.values.tolist()
tested_ts = county_tested.values.tolist()

In [17]:
# Get first case date
county_cases['dt_first_case'] = (county_cases > 0).idxmax(axis=1)
county_cases.loc[county_cases.iloc[:, -2] <= 0, 'dt_first_case'] = np.nan
county_deaths['dt_first_death'] = (county_deaths > 0).idxmax(axis=1)
county_deaths.loc[county_deaths.iloc[:, -2] <= 0, 'dt_first_death'] = np.nan

In [18]:
county_cases['cases_ts'] = cases_ts
county_deaths['deaths_ts'] = deaths_ts

In [19]:
# Add today_case and today_new_case columns
county_cases['today_case'] = county_cases[dt_today]
county_cases['today_new_case'] = county_cases[dt_today] - county_cases[dt_yesterday]
county_deaths['today_death'] = county_deaths[dt_today]
county_deaths['today_new_death'] = county_deaths[dt_today] - county_deaths[dt_yesterday]
county_tested['today_tested'] = county_tested[dt_today]
county_tested['today_new_tested'] = county_tested[dt_today] - county_tested[dt_yesterday]

In [20]:
# Generate reports
case_report = county_cases[['cases_ts','dt_first_case','today_case','today_new_case']]
death_report = county_deaths[['deaths_ts','dt_first_death','today_death','today_new_death']]
tested_report = county_tested[['today_tested','today_new_tested']]
county_report = case_report.join(death_report, how="outer").join(tested_report, how="outer")

In [21]:
# Reset index for county report
county_report = county_report.reset_index()
county_report.columns = ['NAME','cases_ts','dt_first_case','today_case','today_new_case','deaths_ts','dt_first_death','today_death','today_new_death','today_tested','today_new_tested']
county_report['cases_ts'] = county_report['cases_ts'].apply(lambda x: ','.join(map(str,x)))
county_report['deaths_ts'] = county_report['deaths_ts'].apply(lambda x: ','.join(map(str,x)))

In [22]:
# Illinois, Out of State and Suburban Cook is not in geometry
np.setdiff1d(county_report['NAME'],county_geometry['id'])

array([], dtype=object)

In [23]:
# Remove 'Out Of State' and 'Suburban Cook' from county report
county_report = county_report[(county_report['NAME'] != 'Out Of State') & (county_report['NAME'] != 'Suburban Cook')]

In [24]:
# Add start and end dates
county_report['start'] = dt_first
county_report['end'] = dt_today
county_report['dt_unit'] = 'day'
county_report

,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit
0,Adams,"0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,8,9,11...",2020-03-20,13533,22,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-01,157,0,257428,203,2020-03-17,2021-11-08,day
1,Alexander,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-16,769,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-03,13,0,9398,8,2020-03-17,2021-11-08,day
2,Bond,"0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,3,3,3,3,3,3,3,...",2020-03-29,2632,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-16,27,0,51062,40,2020-03-17,2021-11-08,day
3,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,8134,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,84,0,93570,141,2020-03-17,2021-11-08,day
4,Brown,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-01,1087,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-11-12,11,0,41569,5,2020-03-17,2021-11-08,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Whiteside,"1,1,1,1,1,1,1,1,1,1,2,2,3,4,5,5,6,9,15,18,18,2...",2020-03-17,8508,7,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...",2020-04-02,187,0,118856,230,2020-03-17,2021-11-08,day
100,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,93883,119,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1144,0,1548383,2936,2020-03-17,2021-11-08,day
101,Williamson,"0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,2,3,3,3,8,9,...",2020-03-19,12274,7,"0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-19,181,0,176000,212,2020-03-17,2021-11-08,day
102,Winnebago,"1,1,1,2,4,5,5,5,7,8,8,9,12,13,15,18,24,28,36,4...",2020-03-17,43056,100,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,3,...",2020-04-01,567,0,634073,1384,2020-03-17,2021-11-08,day


In [25]:
# Merge geometry with county reports
county_final_gpd = pd.merge(county_geometry, county_report, how="left", left_on="id", right_on="NAME")
county_final_gpd

,id,population,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit
0,Chicago,2693976,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,331031,334,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,5922,2,7028801,14916,2020-03-17,2021-11-08,day
1,McHenry,308570,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,36046,62,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,336,0,571324,1357,2020-03-17,2021-11-08,day
2,Boone,53577,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,8134,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,84,0,93570,141,2020-03-17,2021-11-08,day
3,Ogle,50923,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,7846,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,94,0,96751,144,2020-03-17,2021-11-08,day
4,Will,692310,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,93883,119,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1144,0,1548383,2936,2020-03-17,2021-11-08,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,37620,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,7122,19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,148,0,154183,852,2020-03-17,2021-11-08,day
100,Union,16841,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,3136,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,48,0,63921,78,2020-03-17,2021-11-08,day
101,Pope,4212,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,569,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,6,0,6367,6,2020-03-17,2021-11-08,day
102,Illinois,12671821,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1720345,2060,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,25986,8,36457609,69073,2020-03-17,2021-11-08,day


In [26]:
# Check if NA exists for population
county_final_gpd.loc[county_final_gpd["population"].isna()== True,"id"]

Series([], Name: id, dtype: object)

In [27]:
# Calculate weekly change rates of cases and form a time-series list
county_final_gpd['change_ts'] = ""
for x in range(0, len(county_final_gpd)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = county_final_gpd.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
    county_final_gpd.loc[x,'change_ts'] = ','.join(changeRateStrList)

county_final_gpd

,id,population,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,change_ts
0,Chicago,2693976,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,331031,334,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,5922,2,7028801,14916,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.24,1.44,1.37,0.82,..."
1,McHenry,308570,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,36046,62,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,336,0,571324,1357,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.08,2.33,1.57,0..."
2,Boone,53577,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,8134,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,84,0,93570,141,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4.0,..."
3,Ogle,50923,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,7846,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,94,0,96751,144,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,4...."
4,Will,692310,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,93883,119,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1144,0,1548383,2936,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,9.63,6.69,6.62,4.47,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,37620,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,7122,19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,148,0,154183,852,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,2.0,2.0,3.0,..."
100,Union,16841,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,3136,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,48,0,63921,78,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
101,Pope,4212,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,569,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,6,0,6367,6,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
102,Illinois,12671821,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1720345,2060,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,25986,8,36457609,69073,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.51,2.24,2.24,1.44,..."


In [28]:
# Save file
county_final_gpd.to_file('dph_county_data.geojson', driver='GeoJSON', encoding='utf-8')
print('done')

done


In [29]:
county_final_gpd

,id,population,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,change_ts
0,Chicago,2693976,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,331031,334,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,5922,2,7028801,14916,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.24,1.44,1.37,0.82,..."
1,McHenry,308570,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,36046,62,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,336,0,571324,1357,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.08,2.33,1.57,0..."
2,Boone,53577,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,8134,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,84,0,93570,141,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4.0,..."
3,Ogle,50923,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,7846,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,94,0,96751,144,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,4...."
4,Will,692310,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,93883,119,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1144,0,1548383,2936,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,9.63,6.69,6.62,4.47,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,37620,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,7122,19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,148,0,154183,852,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,2.0,2.0,3.0,..."
100,Union,16841,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,3136,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,48,0,63921,78,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
101,Pope,4212,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,569,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,6,0,6367,6,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
102,Illinois,12671821,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1720345,2060,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,25986,8,36457609,69073,2020-03-17,2021-11-08,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.51,2.24,2.24,1.44,..."
